In [1]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

# Configuration
DATA_ROOTS = [
    r"/home/nigmu/NPersonal/Projects/SDP/nigmu-parkinsons_disease_prediction/Dataset/MDVR",
    r"/home/nigmu/NPersonal/Projects/SDP/nigmu-parkinsons_disease_prediction/Dataset/Italian"
]
BATCH_SIZE = 32
NUM_WORKERS = 4  # Optimize based on CPU
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Custom Dataset Class|
class ParkinsonSpectrogramDataset(Dataset):
    def __init__(self, root_dirs, transform=None):
        if isinstance(root_dirs, str):  # If a single path is given, convert it to a list
            root_dirs = [root_dirs]
        self.root_dirs = root_dirs
        self.transform = transform
        self.samples = self._load_samples()
        
    def _load_samples(self):
        samples = []
        for root_dir in self.root_dirs:
            for class_name in ['HC', 'PD']:
                class_dir = os.path.join(root_dir, class_name)
                if not os.path.exists(class_dir):
                    continue

                # Traverse subfolders inside HC/PD
                for patient_folder in os.listdir(class_dir):
                    patient_path = os.path.join(class_dir, patient_folder)
                    if os.path.isdir(patient_path):  # Ensure it's a directory
                        for img_file in os.listdir(patient_path):
                            if img_file.lower().endswith('.png'):  # Only PNG images
                                img_path = os.path.join(patient_path, img_file)
                                samples.append((img_path, 0 if class_name == 'HC' else 1))

        print(f"✅ Loaded {len(samples)} samples from {self.root_dirs}")
        return samples

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        img = Image.open(img_path).convert('L')  # Convert to grayscale
        if self.transform:
            img = self.transform(img)
        return img, label

# Data Transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)), #Image reduced from 496x200 to 224x224
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Only slight translations
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])  # Normalize between -1 and 1
])


test_transform = transforms.Compose([
    transforms.Resize((224, 224)), #Image reduced from 496x200 to 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

# Create Datasets (Loading from both MDVR & Italian datasets)
train_dataset = ParkinsonSpectrogramDataset([os.path.join(root, 'train') for root in DATA_ROOTS], transform=train_transform)
val_dataset = ParkinsonSpectrogramDataset([os.path.join(root, 'val') for root in DATA_ROOTS], transform=test_transform)
test_dataset = ParkinsonSpectrogramDataset([os.path.join(root, 'test') for root in DATA_ROOTS], transform=test_transform)

# Optimized DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, 
                          num_workers=NUM_WORKERS, pin_memory=True, persistent_workers=True)

val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, 
                        num_workers=NUM_WORKERS, pin_memory=True, persistent_workers=True)

test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, 
                         num_workers=NUM_WORKERS, pin_memory=True)

print(f"✅ DataLoaders ready! Using device: {DEVICE}")

✅ Loaded 154732 samples from ['/home/nigmu/NPersonal/Projects/SDP/nigmu-parkinsons_disease_prediction/Dataset/MDVR/train', '/home/nigmu/NPersonal/Projects/SDP/nigmu-parkinsons_disease_prediction/Dataset/Italian/train']
✅ Loaded 37807 samples from ['/home/nigmu/NPersonal/Projects/SDP/nigmu-parkinsons_disease_prediction/Dataset/MDVR/val', '/home/nigmu/NPersonal/Projects/SDP/nigmu-parkinsons_disease_prediction/Dataset/Italian/val']
✅ Loaded 41886 samples from ['/home/nigmu/NPersonal/Projects/SDP/nigmu-parkinsons_disease_prediction/Dataset/MDVR/test', '/home/nigmu/NPersonal/Projects/SDP/nigmu-parkinsons_disease_prediction/Dataset/Italian/test']
✅ DataLoaders ready! Using device: cuda


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torch.amp import GradScaler, autocast
from torchvision import models
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, precision_recall_curve, average_precision_score
import seaborn as sns
import time
import copy
import gc
import os
from tqdm import tqdm

# Configuration
NUM_EPOCHS = 40
BATCH_SIZE = 32  # Adjust based on your GPU memory
# LEARNING_RATE = 3e-4
LEARNING_RATE = 1e-4  # Reduced from 3e-4
WEIGHT_DECAY = 2e-4
EARLY_STOPPING_PATIENCE = 7
CHECKPOINT_PATH = 'best_densenet_model3.pth'
USE_MIXED_PRECISION = True  # Enable mixed precision training
USE_DATA_AUGMENTATION = True  # Enable more data augmentation
USE_WEIGHTED_LOSS = True  # Handle class imbalance

# Model definition
def create_densenet_model(num_classes=2, dropout_rate=0.3):
    # Using DenseNet121 as base model
    model = models.densenet121(weights='DEFAULT')
    
    # Modify first conv layer to accept grayscale images (1 channel)
    first_conv = model.features.conv0
    model.features.conv0 = nn.Conv2d(
        1, 64, kernel_size=7, stride=2, padding=3, bias=False
    )
    # Initialize with weight averaging from the pre-trained 3-channel weights
    with torch.no_grad():
        model.features.conv0.weight.data = first_conv.weight.data.sum(dim=1, keepdim=True)
    
    # Replace classifier with more robust version
    in_features = model.classifier.in_features
    model.classifier = nn.Sequential(
        nn.BatchNorm1d(in_features),  # Add batch normalization for better stability
        nn.Dropout(dropout_rate),  # Increased dropout for regularization
        nn.Linear(in_features, 512),  # Additional FC layer
        # nn.ReLU(inplace=True),
        # nn.BatchNorm1d(512),
        # nn.Dropout(dropout_rate * 0.8),  # Slightly reduced dropout
        nn.Linear(512, num_classes)
    )

    # model.classifier = nn.Linear(in_features, num_classes)  # Simple linear layer
    
    return model

# Trainer class
class ParkinsonsTrainer:
    def __init__(self, model, train_loader, val_loader, test_loader, device, 
                 criterion=None, learning_rate=3e-4, 
                 weight_decay=2e-4, checkpoint_path='best_model3.pth',
                 class_weights=None, use_mixed_precision=True):
        self.model = model.to(device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader
        self.device = device
        
        # Set criterion based on class weights if provided
        if criterion is None:
            if class_weights is not None and USE_WEIGHTED_LOSS:
                weights = torch.tensor(class_weights).to(device)
                self.criterion = nn.CrossEntropyLoss(weight=weights)
                print(f"Using weighted CrossEntropyLoss with weights: {weights}")
            else:
                self.criterion = nn.CrossEntropyLoss()
        else:
            self.criterion = criterion
            
        # Use AdamW optimizer (better weight decay handling)
        self.optimizer = optim.AdamW(
            model.parameters(), 
            lr=learning_rate, 
            weight_decay=weight_decay,
            betas=(0.9, 0.999)
        )
        
        # # Use cosine annealing scheduler with warm restarts
        # self.scheduler = CosineAnnealingWarmRestarts(
        #     self.optimizer, 
        #     T_0=5,  # Restart every 5 epochs
        #     T_mult=2,  # Double the restart interval after each restart
        #     eta_min=1e-6  # Minimum learning rate
        # )
        
        # Modify scheduler
        self.scheduler = CosineAnnealingWarmRestarts(
            self.optimizer, 
            T_0=3,  # Shorter initial period
            T_mult=2,
            eta_min=1e-6
        )
        
        self.checkpoint_path = checkpoint_path
        self.best_val_acc = 0.0
        self.best_val_f1 = 0.0
        self.early_stopping_counter = 0
        self.use_mixed_precision = use_mixed_precision
        
        # Initialize gradient scaler for mixed precision training
        self.scaler = GradScaler() if use_mixed_precision else None
        
        # Create directory for checkpoints if it doesn't exist
        os.makedirs(os.path.dirname(checkpoint_path) if os.path.dirname(checkpoint_path) else '.', exist_ok=True)
        
    def train_one_epoch(self):
        self.model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        epoch_start = time.time()
        
        # Use tqdm for progress tracking
        pbar = tqdm(self.train_loader, desc="Training")
        
        for batch_idx, (inputs, targets) in enumerate(pbar):

            data_time = time.time()
            
            inputs, targets = inputs.to(self.device), targets.to(self.device)
            
            # Zero gradients
            self.optimizer.zero_grad()
            
            if self.use_mixed_precision:
                # Mixed precision forward pass
                with autocast(device_type='cuda'):
                    outputs = self.model(inputs)
                    loss = self.criterion(outputs, targets)
                
                # Scale loss and backward pass
                self.scaler.scale(loss).backward()

                torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
                
                self.scaler.step(self.optimizer)
                self.scaler.update()
            else:
                # Standard precision
                outputs = self.model(inputs)
                loss = self.criterion(outputs, targets)
                loss.backward()

                torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
                
                self.optimizer.step()
            
            # # Statistics
            # running_loss += loss.item()
            # _, predicted = outputs.max(1)
            # total += targets.size(0)
            # correct += predicted.eq(targets).sum().item()
            
            # Statistics
            if not torch.isnan(loss):
                running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            
            batch_time = time.time() - data_time
            
            # Update progress bar
            pbar.set_postfix({
                'loss': running_loss/(batch_idx+1),
                'acc': 100.*correct/total,
                'time/batch': f"{batch_time:.2f}s"
            })
            
            # Clear memory every 10 batches
            if batch_idx % 10 == 0:
                del inputs, targets, outputs
                torch.cuda.empty_cache()
                gc.collect()

        epoch_time = time.time() - epoch_start
        print(f"Epoch completed in {epoch_time:.2f}s ({epoch_time/60:.2f}min)")
        
        epoch_loss = running_loss / len(self.train_loader) if len(self.train_loader) > 0 else float('inf')
        epoch_acc = 100. * correct / total if total > 0 else 0
    
        return epoch_loss, epoch_acc
    
    def validate(self):
        self.model.eval()
        running_loss = 0.0
        correct = 0
        total = 0
        all_preds = []
        all_targets = []
        
        with torch.no_grad():
            for inputs, targets in tqdm(self.val_loader, desc="Validating"):
                inputs, targets = inputs.to(self.device), targets.to(self.device)
                
                # Forward pass
                if self.use_mixed_precision:
                    with autocast(device_type='cuda'):
                        outputs = self.model(inputs)
                        loss = self.criterion(outputs, targets)
                else:
                    outputs = self.model(inputs)
                    loss = self.criterion(outputs, targets)
                
                # Statistics
                running_loss += loss.item()
                _, predicted = outputs.max(1)
                total += targets.size(0)
                correct += predicted.eq(targets).sum().item()
                
                # Store for F1 score calculation
                all_preds.extend(predicted.cpu().numpy())
                all_targets.extend(targets.cpu().numpy())
                
                # Clear memory
                del inputs, targets, outputs
                
        torch.cuda.empty_cache()
        gc.collect()
        
        val_loss = running_loss / len(self.val_loader)
        val_acc = 100. * correct / total
        
        # Calculate F1 score for validation
        from sklearn.metrics import f1_score
        val_f1 = f1_score(all_targets, all_preds, average='weighted')
        
        return val_loss, val_acc, val_f1
    
    def save_checkpoint(self, is_best=True):
        state = {
            'model': self.model.state_dict(),
            'optimizer': self.optimizer.state_dict(),
            'scheduler': self.scheduler.state_dict(),
            'best_val_acc': self.best_val_acc,
            'best_val_f1': self.best_val_f1
        }
        
        if is_best:
            print(f"Saving best checkpoint with validation accuracy: {self.best_val_acc:.2f}%, F1: {self.best_val_f1:.4f}")
            torch.save(state, self.checkpoint_path)
        
        # Also save periodic checkpoint every 5 epochs
        epoch = state.get('epoch', 0)
        if epoch % 5 == 0:
            torch.save(state, f'checkpoint_epoch_{epoch}.pth')
    
    def train(self, num_epochs, early_stopping_patience=7):
        start_time = time.time()
        train_losses, train_accs = [], []
        val_losses, val_accs, val_f1s = [], [], []
        
        for epoch in range(num_epochs):
            print(f"\nEpoch {epoch+1}/{num_epochs}")
            print('-' * 60)
            
            # Train
            train_loss, train_acc = self.train_one_epoch()
            train_losses.append(train_loss)
            train_accs.append(train_acc)
            
            # Step the scheduler
            self.scheduler.step()
            current_lr = self.optimizer.param_groups[0]['lr']
            print(f"Current learning rate: {current_lr:.8f}")
            
            # Validate
            val_loss, val_acc, val_f1 = self.validate()
            val_losses.append(val_loss)
            val_accs.append(val_acc)
            val_f1s.append(val_f1)
            
            # Print epoch results
            print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
            print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%, Val F1: {val_f1:.4f}")
            
            # Save checkpoint if validation metrics improve
            is_best = False
            if val_acc > self.best_val_acc or (val_acc == self.best_val_acc and val_f1 > self.best_val_f1):
                self.best_val_acc = val_acc
                self.best_val_f1 = val_f1
                is_best = True
                self.save_checkpoint(is_best=True)
                self.early_stopping_counter = 0
            else:
                self.early_stopping_counter += 1
                print(f"EarlyStopping counter: {self.early_stopping_counter} out of {early_stopping_patience}")
                
                if self.early_stopping_counter >= early_stopping_patience:
                    print("Early stopping triggered")
                    break
        
        end_time = time.time()
        training_time = end_time - start_time
        hours, remainder = divmod(training_time, 3600)
        minutes, seconds = divmod(remainder, 60)
        print(f"Training completed in {int(hours)}h {int(minutes)}m {int(seconds)}s")
        
        # Plot training/validation metrics
        self.plot_training_metrics(train_losses, val_losses, train_accs, val_accs, val_f1s)
        
        return train_losses, train_accs, val_losses, val_accs, val_f1s
    
    def plot_training_metrics(self, train_losses, val_losses, train_accs, val_accs, val_f1s=None):
        plt.figure(figsize=(18, 6))
        
        # Plot losses
        plt.subplot(1, 3, 1)
        plt.plot(train_losses, label='Train Loss')
        plt.plot(val_losses, label='Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('Training and Validation Loss')
        plt.legend()
        
        # Plot accuracies
        plt.subplot(1, 3, 2)
        plt.plot(train_accs, label='Train Accuracy')
        plt.plot(val_accs, label='Validation Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy (%)')
        plt.title('Training and Validation Accuracy')
        plt.legend()
        
        # Plot F1 scores if available
        if val_f1s:
            plt.subplot(1, 3, 3)
            plt.plot(val_f1s, label='Validation F1 Score', color='green')
            plt.xlabel('Epoch')
            plt.ylabel('F1 Score')
            plt.title('Validation F1 Score')
            plt.legend()
        
        plt.tight_layout()
        plt.savefig('training_metrics.png', dpi=300)
        plt.close()
    
    def load_best_model(self):
        try:
            checkpoint = torch.load(self.checkpoint_path)
            self.model.load_state_dict(checkpoint['model'])
            print(f"Loaded best model from {self.checkpoint_path}")
            return True
        except Exception as e:
            print(f"Could not load checkpoint: {e}. Using current model state.")
            return False
    
    def test(self, save_predictions=True):
        # Load best model for testing
        self.load_best_model()
        self.model.eval()
        
        all_preds = []
        all_targets = []
        all_probs = []
        
        with torch.no_grad():
            for inputs, targets in tqdm(self.test_loader, desc="Testing"):
                inputs, targets = inputs.to(self.device), targets.to(self.device)
                
                # Forward pass
                if self.use_mixed_precision:
                    with autocast(device_type='cuda'):
                        outputs = self.model(inputs)
                else:
                    outputs = self.model(inputs)
                
                probs = nn.Softmax(dim=1)(outputs)
                
                # Get predictions
                _, preds = outputs.max(1)
                
                # Store for evaluation
                all_preds.extend(preds.cpu().numpy())
                all_targets.extend(targets.cpu().numpy())
                all_probs.extend(probs[:, 1].cpu().numpy())  # Store probability of class 1 (PD)
                
                # Clear memory
                del inputs, targets, outputs
        
        torch.cuda.empty_cache()
        
        # Convert to numpy arrays
        all_preds = np.array(all_preds)
        all_targets = np.array(all_targets)
        all_probs = np.array(all_probs)
        
        # Calculate metrics
        test_acc = 100. * np.mean(all_preds == all_targets)
        print(f"Test Accuracy: {test_acc:.2f}%")
        
        # Classification report
        print("\nClassification Report:")
        report = classification_report(all_targets, all_preds, 
                                      target_names=['Healthy Control', 'Parkinson\'s Disease'])
        print(report)
        
        # Confusion Matrix
        cm = confusion_matrix(all_targets, all_preds)
        self.plot_confusion_matrix(cm)
        
        # ROC Curve
        self.plot_roc_curve(all_targets, all_probs)
        
        # Precision-Recall Curve
        self.plot_pr_curve(all_targets, all_probs)
        
        # Save predictions if requested
        if save_predictions:
            np.savez('test_predictions.npz', 
                    predictions=all_preds,
                    targets=all_targets,
                    probabilities=all_probs)
        
        return test_acc, all_preds, all_targets, all_probs
    
    def plot_confusion_matrix(self, cm):
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                   xticklabels=['Healthy Control', 'Parkinson\'s Disease'],
                   yticklabels=['Healthy Control', 'Parkinson\'s Disease'])
        plt.xlabel('Predicted Label')
        plt.ylabel('True Label')
        plt.title('Confusion Matrix')
        plt.tight_layout()
        plt.savefig('confusion_matrix.png', dpi=300)
        plt.close()
    
    def plot_roc_curve(self, y_true, y_score):
        fpr, tpr, _ = roc_curve(y_true, y_score)
        roc_auc = auc(fpr, tpr)
        
        plt.figure(figsize=(10, 8))
        plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.3f})')
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic (ROC) Curve')
        plt.legend(loc='lower right')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig('roc_curve.png', dpi=300)
        plt.close()
    
    def plot_pr_curve(self, y_true, y_score):
        precision, recall, _ = precision_recall_curve(y_true, y_score)
        pr_auc = average_precision_score(y_true, y_score)
        
        plt.figure(figsize=(10, 8))
        plt.plot(recall, precision, color='green', lw=2, 
                label=f'Precision-Recall curve (AP = {pr_auc:.3f})')
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title('Precision-Recall Curve')
        plt.legend(loc='lower left')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig('pr_curve.png', dpi=300)
        plt.close()

# Memory optimization functions
def optimize_memory():
    # Empty CUDA cache
    torch.cuda.empty_cache()
    # Collect garbage
    gc.collect()

# Main execution
def main():
    # Set device
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Check available memory before starting
    if torch.cuda.is_available():
        print(f"GPU: {torch.cuda.get_device_name(0)}")
        print(f"Total GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
        print(f"Available GPU memory: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB used, "
              f"{(torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_allocated(0)) / 1e9:.2f} GB free")

    
    # Load your data loaders here
    # Assuming train_loader, val_loader, test_loader are defined elsewhere
    # If they're not available, replace with placeholders
    try:
        print("Loading data loaders...")
        # train_loader, val_loader, test_loader should be defined here or imported
        assert 'train_loader' in globals(), "train_loader not defined"
        assert 'val_loader' in globals(), "val_loader not defined"
        assert 'test_loader' in globals(), "test_loader not defined"
    except Exception as e:
        print(f"Warning: {e}")
        print("You'll need to define your data loaders before running this script")
        
    # Calculate class weights for balanced loss (if applicable)
    class_weights = None
    if USE_WEIGHTED_LOSS:
        try:
            # This is a simplified approach - adjust based on your actual class distribution
            # You might need to iterate through your dataset to get accurate counts
            num_healthy = 18834  # Based on your test set statistics
            num_parkinsons = 23052
            total = num_healthy + num_parkinsons
            
            # Calculate inverse frequency
            weight_healthy = total / (2 * num_healthy)
            weight_parkinsons = total / (2 * num_parkinsons)
            
            class_weights = [weight_healthy, weight_parkinsons]
            print(f"Calculated class weights: {class_weights}")
        except:
            print("Could not calculate class weights, using unweighted loss")
    
    # Create model
    model = create_densenet_model(num_classes=2, dropout_rate=0.3)
    print("Model created")
    
    # Initialize trainer
    trainer = ParkinsonsTrainer(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        test_loader=test_loader,
        device=DEVICE,
        learning_rate=LEARNING_RATE,
        weight_decay=WEIGHT_DECAY,
        checkpoint_path=CHECKPOINT_PATH,
        class_weights=class_weights,
        use_mixed_precision=USE_MIXED_PRECISION
    )
    print("Trainer initialized")
    
    # Train the model
    print("\nStarting training...")
    trainer.train(num_epochs=NUM_EPOCHS, early_stopping_patience=EARLY_STOPPING_PATIENCE)
    
    # Test the model
    print("\nEvaluating on test set...")
    test_acc, all_preds, all_targets, all_probs = trainer.test()
    
    print("Complete! Check the saved model and evaluation outputs.")

if __name__ == "__main__":
    # Set seed for reproducibility
    torch.manual_seed(42)
    np.random.seed(42)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(42)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
    # Run main function
    try:
        main() 
    except RuntimeError as e:
        if 'out of memory' in str(e):
            print("\n🚨 CUDA OUT OF MEMORY ERROR")
            print("Try one of these solutions:")
            print("1. Reduce batch size (e.g., BATCH_SIZE = 16 or 8)")
            print("2. Disable mixed precision training (USE_MIXED_PRECISION = False)")
            print("3. Use a smaller DenseNet variant")
            print("4. Resize images to smaller dimensions")
            optimize_memory()
        else:
            raise e

GPU: NVIDIA GeForce GTX 1650
Total GPU memory: 3.90 GB
Available GPU memory: 0.00 GB used, 3.90 GB free
Loading data loaders...
Calculated class weights: [1.111978337050016, 0.9085111920874545]
Model created
Using weighted CrossEntropyLoss with weights: tensor([1.1120, 0.9085], device='cuda:0')
Trainer initialized

Starting training...

Epoch 1/40
------------------------------------------------------------


Training:   1%| | 42/4836 [01:07<2:08:27,  1.61s/it, loss=0.605, acc=66.4, time/

KeyboardInterrupt

